# Código 06: Circulos

Para formar o quadrado, usaremos primitivas de triângulo, mas com os pontos espaçados como quadrado.

### Inicialização do glfw e criação da janela

In [336]:
import math

In [337]:
#Bibliotecas
!pip install glfw
import glfw
!pip install pyopengl
from OpenGL.GL import *
import OpenGL.GL.shaders #Não é redundante?
!pip install numpy
import numpy as np

#Sistema glfw
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)
window = glfw.create_window(720, 600, "Linhas", None, None)
glfw.make_context_current(window)

### Funções de manipulação de matrizes

In [338]:
def multiplica_matrizes(M1, M2):
    
    # recuperando dimensoes de M1
    m1_linhas = len(M1)
    m1_colunas = len(M1[0])
    
    # recuperando dimensoes de M2
    m2_linhas = len(M2)
    m2_colunas = len(M2[0])
    
    
    
    if m1_colunas != m2_linhas:
        print(m1_linhas,m1_colunas,m2_linhas,m2_colunas)
        print('Nao posso multiplicar. Dimensoes incorretas.')
        return -1

    # criando espaco para a M3
    M3 = [[0 for row in range(m2_colunas)] for col in range(m1_linhas)]
    
    for i in range(m1_linhas):
        for j in range(m2_colunas):
            for k in range(m1_colunas):
                M3[i][j] += M1[i][k] * M2[k][j]
                
    for i in range(m1_linhas):
        for j in range(m2_colunas):
            M3[i][j] = int(M3[i][j])
                
    return M3

def translacao(vertice, t_x, t_y):
    
    # define a matriz de translacao
    matriz_translacao = [
                            [1, 0, t_x],
                            [0, 1, t_y],
                            [0, 0, 1  ]
                        ]
    
    # inicializa novo vertice
    vertice_t = multiplica_matrizes(matriz_translacao, vertice)
    
    # retorna novo vertice
    return vertice_t

def escala(vertice, e_x, e_y ):
    
    # define a matriz de translacao
    matriz_escala = [
                            [e_x, 0  , 0  ],
                            [0  , e_y, 0  ],
                            [0  , 0  , 1  ]
                        ]
    
    # inicializa novo vertice
    vertice_e = multiplica_matrizes(matriz_escala, vertice)
    
    # retorna novo vertice
    return vertice_e

def rotacao(vertice, angulo):

    rad = math.radians(angulo) 
    c = math.cos(rad)
    s = math.sin(rad)
    
    # define a matriz de rotacao 
    matriz_rotacao = [
                        [c  , -s , 0],
                        [s  , c  , 0],
                        [0  , 0  , 1]
                     ]
    

    
    # inicializa novo vertice
    vertice_r = multiplica_matrizes(matriz_rotacao, vertice)
    
    return vertice_r

### Eventos de teclado e mouse

In [339]:
t_x = 0
t_y = 0

def key_event(window,key,scancode,action,mods):
    global t_x, t_y
    
    print('[key event] key=',key)
    print('[key event] scancode=',scancode)
    print('[key event] action=',action)
    print('[key event] mods=',mods)
    print('-------')
    if key == 265: t_y += 0.01 #cima
    if key == 264: t_y -= 0.01 #baixo
    if key == 263: t_x -= 0.01 #esquerda
    if key == 262: t_x += 0.01 #direita

glfw.set_key_callback(window,key_event)

def mouse_event(window,button,action,mods):
    print('[mouse event] button=',button)
    print('[mouse event] action=',action)
    print('[mouse event] mods=',mods)
    print('-------')
    
glfw.set_mouse_button_callback(window,mouse_event)

### Shaders: código, espaço, compilação e linkagem

In [340]:
#GLSL para Vertex Shader
vertex_code = """
        attribute vec2 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,0.0,1.0);
        }
        """
#GLSL para Fragment Shader
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

#Requisitando slot para GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

#Associando os códigos aos espaços
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

#Compilando shader de vértice
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

#Compilando shader de fragmento
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

#Associadno programas compilados ao programa principal
glAttachShader(program, vertex)
glAttachShader(program, fragment)

#Linkagem do programa
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
#Tornando programa o atual
glUseProgram(program)

### Criação dos vértices

In [341]:
#Criando espaço
vertices = np.zeros(3, [("position", np.float32, 2)])

#Preenchendo as coordenadas
vertices['position'] = [
                            (-0.5, 0),
                            (+0.5, 0),
                            (0, 0.7)
                        ]

### Manipulação dos espaços de dados

In [342]:
#Requisitando espaço de buffer para GPU
buffer = glGenBuffers(1)
#Tornando o buffer o buffer padrão de dados
glBindBuffer(GL_ARRAY_BUFFER, buffer)
#Subindo os dados de vértice para o buffer na GPU
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
#glBindBuffer(GL_ARRAY_BUFFER, buffer)

#Encontrando informações de stride e offset dos vértices
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
#Capturando posição do atributo "position" e habilitando
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)
#Linkando dados ao atributo "position"
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

### Alterando cor

In [343]:
loc_color = glGetUniformLocation(program, "color")
R = 1.0
G = 0.0
B = 0.0

### Exibindo na tela

In [344]:
glfw.show_window(window)

### Loop principal

In [345]:
t_y = 0
t_x = 0
while not glfw.window_should_close(window):

    glfw.poll_events() 
    # if t_y != 1 or t_x != t_x:
    #     print(t_y)
    #     print(t_x)
    
    glClear(GL_COLOR_BUFFER_BIT) 
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    mat_translation = np.array([    1.0, 0.0, 0.0, t_x, 
                                    0.0, 1.0, 0.0, t_y, 
                                    0.0, 0.0, 1.0, 0.0, 
                                    0.0, 0.0, 0.0, 1.0], np.float32)

    s_x = 1
    s_y = 1
    mat_scale       = np.array([    s_x, 0.0, 0.0, 0.0, 
                                    0.0, s_y, 0.0, 0.0, 
                                    0.0, 0.0, 1.0, 0.0, 
                                    0.0, 0.0, 0.0, 1.0], np.float32)
    
    angulo = 0
    c = math.cos(angulo)
    s = math.sin(angulo)

    mat_rotation = np.array([       c,   -s,  0.0, 0.0, 
                                    s,    c,  0.0, 0.0, 
                                    0.0, 0.0, 1.0, 0.0, 
                                    0.0, 0.0, 0.0, 1.0], np.float32)
    
    mat_final = angulo*mat_scale*mat_translation
    
    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_final)

    glDrawArrays(GL_TRIANGLES, 0, len(vertices))
    glUniform4f(loc_color, R, G, B, 1.0) ### modificando a cor do objeto!

    glfw.swap_buffers(window)

glfw.terminate()

[mouse event] button= 0
[mouse event] action= 1
[mouse event] mods= 0
-------
[mouse event] button= 0
[mouse event] action= 0
[mouse event] mods= 0
-------
[key event] key= 263
[key event] scancode= 331
[key event] action= 1
[key event] mods= 0
-------
[key event] key= 263
[key event] scancode= 331
[key event] action= 2
[key event] mods= 0
-------
[key event] key= 263
[key event] scancode= 331
[key event] action= 2
[key event] mods= 0
-------
[key event] key= 263
[key event] scancode= 331
[key event] action= 2
[key event] mods= 0
-------
[key event] key= 263
[key event] scancode= 331
[key event] action= 2
[key event] mods= 0
-------
[key event] key= 263
[key event] scancode= 331
[key event] action= 2
[key event] mods= 0
-------
[key event] key= 263
[key event] scancode= 331
[key event] action= 2
[key event] mods= 0
-------
[key event] key= 263
[key event] scancode= 331
[key event] action= 2
[key event] mods= 0
-------
[key event] key= 263
[key event] scancode= 331
[key event] action= 2